### Scraping for news headlines on Reuters Website

In [10]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import dateutil.parser
import time
import csv
from datetime import datetime
import io
import os
import pandas as pd
import numpy as np

In [11]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # Set counter
    x = 1
    
    while (x <= 5000):

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        x += 1
        
        # Assess conditions
        if new_height == last_height or x == 5000:            
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [12]:
def fetch_news(url):
    
    # Setup driver
    driver = webdriver.Chrome('chromedriver.exe')
    
    # Wait for 30sec before throwing any exceptions
    driver.implicitly_wait(30)
    
    # Open web page
    driver.get(url)
    
    # Start scrolling through pages
    scroll(driver, 5)
    
    # Assign located element to a variable
    stockNews = driver.find_elements_by_xpath('//div[@class="item"]')
    
    # Once scroll returns bs4 parsers the page_source
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    itemList=soup.find_all('div', attrs={"class":"item"})
        
    # Close driver after page sourcing
    #driver.close()
    
    # Empty arrays to store news and dates    
    stockNewsLines = []
    #stockNewsPeriod = []
    stockNewsLinks = []
    stockNewsDates = []
    
    # Loop through all elements in the page source   
    for s in range(len(stockNews)):
        stockNewsLines.append(stockNews[s].text.partition("\n")[0])
        #stockNewsPeriod.append(stockNews[s].text.partition("\n")[2].partition("\n")[2])
     
    
    # Open news article on new tab
    for i in range(len(itemList)):
        #stockNewsLinks.append(itemList[i].find('a').get('href'))

        # Open a new window
        driver.execute_script("window.open('');")

        # Switch to the new window and open URL B
        driver.switch_to.window(driver.window_handles[1])
        driver.get(itemList[i].find('a').get('href'))

        # Assign located element to a variable
        articleTimeStamp = driver.find_elements_by_xpath('//*[@id="__next"]/div/div[3]/div[2]/div/div/div/div/time[1]')

        for a in range(len(articleTimeStamp)):
            stockNewsDates.append(articleTimeStamp[a].text)

        # Close the tab with URL B
        driver.close()

        # Switch back to the first tab with URL A
        driver.switch_to.window(driver.window_handles[0]) 
        
    
    # Organise the collected data in a table
    newsTable = pd.DataFrame({'Date':stockNewsDates,'News':stockNewsLines})
    
    # Check if stock news data exit
    if not os.path.exists('./msft_news.csv'):

    # make directory and store stock news    
        #os.makedirs('stockdata')
        newsTable.to_csv('./msft_news.csv', index=False)
    else:
        pass
    
    # Return results in data frame
    return newsTable

In [13]:
# Asssign a reuters url for Dow Jones Index
tr_url = 'https://www.reuters.com/companies/MSFT.OQ' #'https://www.reuters.com/quote/.DJI'

# Perform function
fetch_news(tr_url)

,Date,News
0,"OCTOBER 1, 2020",Amazon and Big Tech cozy up to Biden camp with...
1,"SEPTEMBER 30, 2020",Microsoft raises reward for outside law firms ...
2,"SEPTEMBER 29, 2020","Microsoft says disruption to Teams, Outlook re..."
3,"SEPTEMBER 29, 2020",Microsoft 365 outage affects multiple services
4,"SEPTEMBER 28, 2020",Microsoft gets into 5G race with Azure cloud f...
...,...,...
527,"SEPTEMBER 20, 2017","BRIEF-Dell EMC, General Dynamics and Microsoft..."
528,"SEPTEMBER 20, 2017",BRIEF-Essity and Microsoft partner on Internet...
529,"SEPTEMBER 19, 2017",BRIEF-Microsoft announces quarterly dividend a...
530,"SEPTEMBER 18, 2017","Microsoft's Hotmail, Outlook.com services back..."


In [ ]:
newsLink = pd.read_csv('./msft_news.csv', usecols=['Link'])
newsLink